# Lista 6

## Capítulo 9, exercício 4

Describe the main components necessary to add to a "standard" EA in order to tackle a multiobjective problem.

    É necessário que o EA seja capaz de efetuar as etapas do processo evolucionário. Etapas como a selação do pais, que dependem da fitness dos indivíduos devem ser modificadas para levar em consideração todos os objetivos. O mesmo vale para a etapa de seleção de sobreviventes, que ão pode ser beaseada em regras que dependem de um único valor para a fitness. Maneiras de fazer isso incluem a escalarização, que é a combinação de todos os objetivos em uma única função objetivo, atribuíndo pesos a cada função objetivo particular, ou usar um dos vários métodos propostos no capítulo, como por exemplo o MOGA (Fonesa e Flaming) que atribui a cada indivíduo um valor artificial para a fitness igual à quantidade de indivíduos da população atual que são dominados por este.
    Além de tratar as etapas do algoritmo que lidam diretamente com o valor da fitness, é necessário levar em consideração que enquanto problemas com uma única função objetivo têm somente um único fenótipo ótimo, problemas multiobjetivo têm como fenótipos não-dominados uma "frente de Pareto" potencialmente ilimitada. Como é desejável gerar indivíduos cujos fenótipos cubram a "frente de Pareto", em problema de multiobjetivo é muito importante a manutenção da diversidade na população de um EA.

## Capítulo 10, exercício 2 (só a primeira fase do enunciado).

Implement a simples memetic algorithm using a single iteration of a bit-flipping local search within the code for the SGA you developed for One-Max in Chapter 3.

## Capítulo 12, exercício 1

Specify the eight-queens problems as a CSP $(S, \phi)$

O problema das oito rainhas consiste em encontrar uma lista de oito posições distintas no tabuleiro de xadrez (que podem ser representadas, por exemplo, cada uma por um par ordenado $(n, m) \in \{1, 2, 3, 4, 5, 6, 7, 8\}^{2}$, uma posição para cada rainha, que satisfaça a seguinte restrição: nenhuma rainha deve pôr nenhuma das outras em xeque.

Essa restrição pode ser considerada como um conjunto de $8$ restrições, uma para cada rainha, a $i$-ésima restrição seria: A $i$-ésima rainha não deve pôr nenhuma das outras em xeque. Ou, um conjunto de $28$ restrições, uma para cada par de rainhas, seriam restrições do tipo: A $i$-ésima e a $j$-ésima rainhas não devem estar mutuamente em xeque.

Temos então: $S =$ conjunto de todas as listas de $8$ posições distintas dentre as $64$ de um tabuleiro de xadrez, e dada uma lista $L = [ L_1, L_2, \dots, L_8] \in S$ de posições das rainhas, $\phi(L)$ será uma: $\phi(L) = $ (as rainhas em $L_1$ e $L_2$ não estão em xeque) $\bigwedge$ (as rainhas em $L_1$ e $L_3$ não estão em xeque) ...  até conferir todas as combinações.


## Questão 4

Escreva (utilizando um pseudo-código) um algoritmo genético simples para a solução do Exercício 4 da Lista de Exercícios #5, considerando que a população não é de dicionários $\mathbf{Y}$, mas sim de matrizes $p_{\mathbf{y}|\mathbf{x}}$ descrevendo as probabilidades com as quais um vetor de dados $\mathbf{x}$ é atribuído a um agrupamento $\mathbf{y}$.

No caso de soft-clustering, onde um vetor de dados pertence, com diferente probabilidades, a vários agrupamentos, os centroides podem ser calculados como:
\begin{equation}
\mathbf{y}_{k} = \frac{\sum_{n=1}^{N}p(\mathbf{y}_k | \mathbf{x}_n)\mathbf{x}_n}{\sum_{n=1}^{N}p(\mathbf{y}_k | \mathbf{x}_n)}
\end{equation}
e o erro quadrático médio é
\begin{equation}
D = \sum_{n=1}^{N}p(\mathbf{x}_n)\sum_{n=1}^{K}p(\mathbf{y}_k|\mathbf{x}_n)||\mathbf{y}_k - \mathbf{x}_n||^2.
\end{equation}
Para vetores de entradas equiprováveis, o erro quadrático médio pode ser escrito como:
\begin{equation}
D = \frac{1}{N} \sum_{n=1}^{N}\sum_{n=1}^{K} p(\mathbf{y}_k | \mathbf{x}_n)||\mathbf{y}_k - \mathbf{x}_n||^{2}
\end{equation}
Com isso podemos escolher um genótipo formado pelas probabilidades condicionais $p(y|x)$ e um fenótipo que apresenta centroides calculados a partir da equação de $\mathbf{y}_{k}$. A função da fitness pode ser, por xemplo escolhida como $F(\mathbf{Y})  -D$, já que $D \geq 0$. Onde $\mathbf{Y}$ é dicionário obtido pela tradução de $g$, genótipo do indivíduo. Quanto a recombinação e mutação, é possível usar crossover de um ponto e perturbação gaussiana. No entanto, como os genes representam probabilidades, devemos normalizar os valores após as operações. Desta forma garantimos que os operadores sempre gerarão indivíduos válidos. 

Então podemos escrever o pseudo-código:

Tamanho da população: $\mu$, População: $\mathbf{P}$, Filhos: $\mathbf{O}$, Probabilidade de recombinação: $p_r$, Desvio-padrão da mutação: $\sigma_m$, Número máximo de avaliações: $L$, Tradução do genótipo para fenótipo: $trad(.)$, Vetor de fitness: $\mathbf{f}$, Função Custo: $D$, Melhor aptidão: best-apt, Melhor candidato: best-cand. 

$\textbf{while}$ i < L $\textbf{do}$:

-- $f_i$ = $D(trad(p_i))$
    
-- $\textbf{if}$ $f_i$ > best-apt

-- -- best-cand = $p_i$ 

-- $\mathbf{M}$ = SUS($\mathbf{P}$)

-- $\mathbf{O}$ = crossover($\mathbf{M}$, $p_r$)

-- $\mathbf{O}$ = $\mathbf{O}$ + $\sigma_r N(0,1)$

-- $\mathbf{O}$ = norm($\mathbf{O}$

-- $\mathbf{P}$ = $\mathbf{O}$

$\mathbf{Y}^{*}$ = t(best-cand)
